In [ ]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


df = pd.read_csv(r"C:\Users\risha\Desktop\capstone project\Coursera\Google analytics\data from previous 12 months\cleaned_combined.csv")
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
df['ride_length_mins']= pd.to_numeric(df['ride_length_mins'],errors='coerce')
df.info()
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4714118 entries, 0 to 4714117
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  ride_length_mins    float64       
dtypes: datetime64[ns](2), float64(5), object(7)
memory usage: 503.5+ MB
          start_lat     start_lng       end_lat       end_lng  \
count  4.714116e+06  4.714116e+06  4.714116e+06  4.714116e+06   
mean   4.190231e+01 -8.764777e+01  4.190256e

In [18]:
import numpy as np
import scipy.stats as st
from scipy.stats import zscore

# std_dev = df['ride_length_mins'].std() 
mean = 16.49233
std_dev = 29.40938

# Define the parameters
confidence_level = 0.95                # siginifucance level of 0.05 for 95% confidence interval and stat power of 0.8 
margin_of_error = 1            #usually 5% for 95% confidence interval as per convention but can vary from +-3% to +-8%
population_size = 47141118

# Calculate the Z-score
df['z_score'] = zscore(df['ride_length_mins'])            #or zscore = st.norm.ppf(1 - (1 - confidence_level) / 2) 


#this formula is only used for nuumerical data. the other one where [z*p*(1-p)]/[moe^2] is used for categorical/binary data
sample_size = np.ceil( z_score**2 * std_dev**2/ margin_of_error**2)

print(f"Z-Score: {z_score:.2f}")
print(f"Required Sample Size: {int(sample_size)}")

Z-Score: 1.96
Required Sample Size: 3323


In [19]:
import numpy as np
import scipy.stats as st

# Step 1: Define parameters
std_dev = 29.40938  # Standard deviation in minutes (assumed; calculate from data if possible)
confidence_level = 0.95
margin_of_error = 1.0  # Adjusted to 1 minute for practicality
population_size = 4714118  # Total rides: 4.714M

# Step 2: Calculate the Z-score
z_score = st.norm.ppf(1 - (1 - confidence_level) / 2)

# Step 3: Calculate the sample size without finite population correction
numerator = z_score**2 * std_dev**2
denominator = margin_of_error**2
initial_sample_size = numerator / denominator

# Step 4: Apply finite population correction
corrected_sample_size = initial_sample_size * population_size / (population_size + initial_sample_size)

# Step 5: Round up to the nearest integer
sample_size = np.ceil(corrected_sample_size)
print(f"Z-Score: {z_score:.2f}")
print(f"Required Sample Size (Total): {int(sample_size)}")

# For subgroup analysis
sample_size_members = np.ceil(sample_size * 0.5809)
sample_size_casuals = np.ceil(sample_size * 0.4191)
print(f"Sample Size for Members: {int(sample_size_members)}")
print(f"Sample Size for Casuals: {int(sample_size_casuals)}")

Z-Score: 1.96
Required Sample Size (Total): 3321
Sample Size for Members: 1930
Sample Size for Casuals: 1392


Hypothesis Testing Metrics

Ride Length (ride_length_mins):
H0: No difference in mean ride length between members and casuals.
Test: Two-sample t-test

In [20]:
members = df.loc[df['member_casual'] == 'member', 'ride_length_mins'].sample(n=1930, random_state=42)
casuals = df.loc[df['member_casual'] == 'casual', 'ride_length_mins'].sample(n=1392, random_state=42)
t_stat, p_val = st.ttest_ind(members, casuals, equal_var=False)
print(f"t-stat: {t_stat}, p-value: {p_val}")

t-stat: -10.690162161547308, p-value: 7.459468105703486e-26


t-statistic: -10.69 indicates a significant difference, with casuals having a higher mean (22.3 min) than members (12.3 min), as the negative value suggests the difference aligns with the expected direction.
p-value: 7.46e-26 (< 0.05) strongly rejects the null hypothesis (H0: no difference), confirming a statistically significant difference in ride lengths.